<a href="https://colab.research.google.com/github/sharop/CD3001B/blob/main/clase_1/Effects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install linearmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.2/68.2 KB 4.6 MB/s eta 0:00:00


In [6]:
# Example 16.4 Panel Data Analysis for Investment Demand
# Deviation Approach

import numpy as np
import pandas as pd
from scipy import stats

# read 5 data files
data1 = pd.read_csv("http://web.pdx.edu/~crkl/ceR/data/ifcgm.txt", sep='\s+', nrows=20)
data2 = pd.read_csv("http://web.pdx.edu/~crkl/ceR/data/ifcch.txt", sep='\s+', nrows=20)
data3 = pd.read_csv("http://web.pdx.edu/~crkl/ceR/data/ifcge.txt", sep='\s+', nrows=20)
data4 = pd.read_csv("http://web.pdx.edu/~crkl/ceR/data/ifcwe.txt", sep='\s+', nrows=20)
data5 = pd.read_csv("http://web.pdx.edu/~crkl/ceR/data/ifcus.txt", sep='\s+', nrows=20)
data1['ID'] = 1
data2['ID'] = 2
data3['ID'] = 3
data4['ID'] = 4
data5['ID'] = 5
# set up panel data
data = pd.concat([data1,data2,data3,data4,data5])
data.set_index(['ID', 'YEAR'], inplace=True)
# switch ID and YEAR to study the time effects
# data.set_index(['YEAR', 'ID'], inplace=True)



* i =  5 empresas: General Motors, Chrysler, General Electric, Westinghouse, and United Steel.
* t = 20 years: 1935-1954.
* $I_it$ = Inversion bruta
* $F_it$ = Valor de mercado
* $C_it$ = Valor del stock de planta y equipo.

In [7]:
data

I       F      C
ID YEAR                      
1  1935  317.6  3078.5    2.8
   1936  391.8  4661.7   52.6
   1937  410.6  5387.1  156.9
   1938  257.7  2792.2  209.2
   1939  330.8  4313.2  203.4
...        ...     ...    ...
5  1950  418.8  1677.4  357.8
   1951  588.2  2289.5  342.1
   1952  645.2  2159.4  444.2
   1953  641.0  2031.3  623.6
   1954  459.3  2115.5  669.7

[100 rows x 3 columns]

In [9]:
# Pooled OLS estimator
from linearmodels import PooledOLS
pooled = PooledOLS.from_formula('I ~ 1 + F + C', data).fit()
print(pooled)



                          PooledOLS Estimation Summary                          
Dep. Variable:                      I   R-squared:                        0.7789
Estimator:                  PooledOLS   R-squared (Between):              0.7725
No. Observations:                 100   R-squared (Within):               0.7929
Date:                Thu, Feb 16 2023   R-squared (Overall):              0.7789
Time:                        20:38:37   Log-likelihood                   -624.99
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      170.81
Entities:                           5   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                    F(2,97)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             170.81
                            

In [10]:
# Between estimator
from linearmodels import BetweenOLS
between = BetweenOLS.from_formula('I ~ 1 + F + C', data).fit()
print(between)



                         BetweenOLS Estimation Summary                          
Dep. Variable:                      I   R-squared:                        0.9082
Estimator:                 BetweenOLS   R-squared (Between):              0.9082
No. Observations:                   5   R-squared (Within):              -12.490
Date:                Thu, Feb 16 2023   R-squared (Overall):             -3.2890
Time:                        20:38:43   Log-likelihood                   -28.111
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      9.8975
Entities:                           5   P-value                           0.0918
Avg Obs:                       20.000   Distribution:                     F(2,2)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             9.8975
                            

In [ ]:
# First differences estimator (without constant term)
from linearmodels import FirstDifferenceOLS
firstdiff = FirstDifferenceOLS.from_formula('I ~ F + C', data).fit()
print(firstdiff)

from linearmodels.panel.results import compare
res1 = {'Pooled':pooled,'Between':between,'firstdiff':firstdiff}
print(compare(res1))



In [ ]:
# Fixed effects or within estimator, constant included
from linearmodels import PanelOLS
fixed = PanelOLS.from_formula('I ~ 1 + F + C + EntityEffects', data).fit()
print(fixed)
# extract fixed effects
fixed.estimated_effects
fixed_effects = fixed.params.Intercept + fixed.estimated_effects.unstack(level=0).values[0]
print(fixed_effects)
# F test for fixed effects versus OLS
print(fixed.f_pooled)



In [ ]:
# Random effects estimator
from linearmodels import RandomEffects
random = RandomEffects.from_formula('I ~ 1 + F + C + EntityEffects', data).fit()
print(random)
# extract fixed effects
random.estimated_effects
random_effects = random.params.Intercept + random.estimated_effects.unstack(level=0).values[0]
print(random_effects)



In [ ]:
# compare fixed effects and random effects models
res2 = {'Pooled':pooled,'Fixed':fixed,'Random':random}
print(compare(res2))



In [ ]:
# compare the estimates of fixed effects and random effects
effects = pd.DataFrame({'Fixed Effects':fixed_effects,'Random Effects':random_effects},
                       index=data.index.levels[0])
print(effects)



In [ ]:
# LM test for random effects versus OLS
n = data.index.levels[0].size
T = data.index.levels[1].size
D = np.kron(np.eye(n), np.ones(T)).T
e = pooled.resids
LM = (e.dot(D).dot(D.T).dot(e) / e.dot(e) - 1) ** 2 * n * T / 2 / (T - 1)
LM_pvalue = stats.chi2(1).sf(LM)
print("LM Test: chisq = {0}, df = 1, p-value = {1}".format(LM, LM_pvalue))



In [ ]:
# Hausman test for fixed versus random effects model
psi = fixed.cov.iloc[1:,1:] - random.cov.iloc[1:,1:]
diff = fixed.params[1:] - random.params[1:]
W = -diff.dot(np.linalg.inv(-psi)).dot(-diff)
dof = random.params.size - 1
pvalue = stats.chi2(dof).sf(W)
print("Hausman Test: chisq = {0}, df = {1}, p-value = {2}".format(W, dof, pvalue))